In [ ]:
from tweepy import OAuthHandler
import tweepy
import json
import pandas as pd
import re
import os
import time
from datetime import datetime


#function defintions
def scrapetweet(search_words, date_since, numTweets):
    j_file = json.load(open('twitter_credentials.json', 'r'))
    consumer_key = j_file['CONSUMER_KEY']
    consumer_secret = j_file['CONSUMER_SECRET']
    access_token = j_file['ACCESS_TOKEN']
    access_secret = j_file['ACCESS_SECRET']

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    
    db_tweets = pd.DataFrame(columns=['username', 'acct_desc', 'location', 'following', 'followers', 'total_tweets', 'user_created', 
                                     'tweet_created', 'retweet_count', 'text', 'hashtags'])
            
    tweets = tweepy.Cursor(api.search, q=search_words, lang='en', since=date_since, tweet_mode='extended').items(numTweets)

    tweet_list = [tweet for tweet in tweets]

    for tweet in tweet_list:
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        total_tweets = tweet.user.statuses_count
        user_created = tweet.user.created_at
        tweet_created = tweet.created_at
        retweet_count = tweet.retweet_count
        hashtags = tweet.entities['hashtags']

        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text

        ith_tweet = [username, acctdesc, location, following, followers, total_tweets, user_created, tweet_created, retweet_count,
                    text, hashtags]

        db_tweets.loc[len(db_tweets)] = ith_tweet    
    
    return db_tweets


def clean_hashtag(hashtag):
    hashtag = hashtag.replace("{'text':", '')
    hashtag = hashtag.replace("'indices': [", '')
    hashtag = hashtag.replace(']', '')
    hashtag = hashtag.replace('[', '')
    hashtag = hashtag.replace('\'', '')
    hashtag = hashtag.replace('}', '')
    
    
    hashtag_list = hashtag.split(',')

    for i in range(len(hashtag_list)):
        hashtag_list[i] = hashtag_list[i].strip()

    for i in hashtag_list:
        if str.isdigit(i):
            hashtag_list.remove(i)

    for i in hashtag_list:
        if str.isdigit(i):
            hashtag_list.remove(i)

    return hashtag_list


def remove_https(desc):
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', desc)
    return text

def strip_emoji(text):
    RE_EMOJI = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', text)

def clean_data(df):
    df.fillna('None', inplace=True)
    df['hashtags'] = df['hashtags'].apply(clean_hashtag)
    df['acct_desc'] = df['acct_desc'].apply(strip_emoji)
    df['acct_desc'] = df['acct_desc'].apply(remove_https)
    return df


#define the hashtags, starting date and number of tweets to get
search_words = "#70daysofML OR #AIbootcamp2020 OR #DSN70daysofML OR #70daysofDL OR #DSN70daysofDL OR #1_million_AI_talents_in_10_year OR #4-day_scorestreak"
date_since = "2020-07-01"
numTweets = 300

#get new tweets and clean it
new_df = scrapetweet(search_words, date_since, numTweets)
new_df = clean_data(new_df)

#then append it to the previous data
df = pd.read_csv('latest.csv')    #replace the path later
df.append(new_df, ignore_index=True)

#the "df" dataframe can now be used for visualization